In [1]:
from datetime import date, datetime, timedelta
from gridfs import GridFS
from pymongo import MongoClient

from ajna_commons.flask.conf import DATABASE, MONGODB_URI
from ajna_commons.utils.images import get_cursor, get_imagens_recortadas

db = MongoClient(host=MONGODB_URI)[DATABASE]
fs = GridFS(db)

2019-03-26 17:24:16,149 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  /home/ivan/pybr/ajna/padma/ajna_commons/flask/error.log
Fazendo log de atividade no arquivo  /home/ivan/pybr/ajna/padma/ajna_commons/flask/access.log


In [2]:
FALTANTES = {'metadata.contentType': 'image/jpeg',
             'metadata.carga.atracacao.escala': None}

DELTA_VAZIO = 5
DELTA_IMPORTACAO = -5
DELTA_EXPORTACAO = +10

def carga_faltantes(data_inicio, data_fim):
    filtro = FALTANTES
    data_fim = data_fim + timedelta(hours=1, minutes=59, seconds=59)  # Pega atá a última hora do dia
    filtro['metadata.dataescaneamento'] = {'$gte': data_inicio, '$lte': data_fim}
    projection = ['metadata.numeroinformado', 'metadata.dataescaneamento']
    print(filtro)
    total = db['fs.files'].count_documents(filtro)
    fs_cursor = db['fs.files'].find(filtro, projection=projection).sort('metadata.numeroinformado')
    return total, fs_cursor

start = datetime(2017, 7, 5)
end = datetime(2017, 7, 15)
total, fs_cursor = carga_faltantes(start, end)

{'metadata.contentType': 'image/jpeg', 'metadata.carga.atracacao.escala': None, 'metadata.dataescaneamento': {'$lte': datetime.datetime(2017, 7, 15, 1, 59, 59), '$gte': datetime.datetime(2017, 7, 5, 0, 0)}}


In [3]:
total

424

In [4]:
fs_faltantes = list(fs_cursor)
set_faltantes = set()
dict_faltantes = dict()
for linha in fs_faltantes:
    numero = linha['metadata']['numeroinformado'].lower()
    dict_faltantes[numero] = linha['_id']
    set_faltantes.add(numero)

In [5]:
AGREGATE_VAZIO = [
    {'$match': {'dataatracacaoiso':
                {'$gte': start  - timedelta(days=DELTA_VAZIO),
                 '$lte': end + timedelta(days=DELTA_VAZIO)}}},
    {'$lookup': {
        'from': 'CARGA.EscalaManifesto',
        'localField': 'escala',
        'foreignField': 'escala',
        'as': 'lista_manifestos'
    }}, 
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'lista_manifestos.manifesto': 1}},
    {'$lookup': {
        'from': 'CARGA.ContainerVazio',
        'localField': 'lista_manifestos.manifesto',
        'foreignField': 'manifesto',
        'as': 'containers_vazios'
    }},
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'containers_vazios.container': 1,
                  'containers_vazios.manifesto': 1}}
]


In [6]:
cursor = db['CARGA.AtracDesatracEscala'].aggregate(AGREGATE_VAZIO)

In [7]:
lista = list(cursor)

In [8]:
len(lista)

311

In [9]:
set_manifestos = set()
dict_manifestos = dict()
for lista2 in lista:
    for linha in lista2['containers_vazios']:
        dict_manifestos[linha['container']] = linha['manifesto']
        set_manifestos.add(linha['container'])
        
len(set_manifestos)

27275

In [10]:
encontrados_vazio = set_faltantes & set_manifestos
encontrados_vazio

{'caiu9703069',
 'crtu7001499',
 'fscu4697269',
 'sudu2837401',
 'sudu2841269',
 'sudu2841551',
 'sudu2841850',
 'sudu2850682',
 'sudu2851523',
 'sudu2861938',
 'tghu6303892',
 'triu0300295',
 'uacu8192695',
 'uacu8226060',
 'uacu8233182',
 'uacu8427450',
 'uacu8556204'}

In [12]:
for container in encontrados_vazio:
    print(dict_faltantes[container])
    print(dict_manifestos[container])

5c530aa21004b377f3dbc88b
2117b01587440
5c530b771004b377f3dc28b5
1517701532957
5c530f001004b377f3dd1c9d
1517501415469
5c530eea1004b377f3dd15d9
1017b01574231
5c530b8c1004b377f3dc2e0d
1517501415469
5c530ee11004b377f3dd1491
1517501415469
5c530eda1004b377f3dd12f9
1517501415469
5c530aa91004b377f3dbd067
2117b01587440
5c530aa91004b377f3dbcfd3
2117b01587440
5c530b731004b377f3dc26a9
1517701532957
5c530aa41004b377f3dbca93
2117b01587440
5c530ec91004b377f3dd0f3d
1517501415469
5c530eff1004b377f3dd1c5d
1517501415469
5c530efe1004b377f3dd1c45
1517501415469
5c530aa31004b377f3dbc99f
2117b01587440
5c530f071004b377f3dd1eb9
1517501415469
5c530ed01004b377f3dd10f1
1517501415469


In [13]:
AGREGATE_IMPORTACAO = [
    {'$match': {'dataatracacaoiso':
                {'$gte': start  - timedelta(days=10),
                 '$lte': end}}},
    {'$lookup': {
        'from': 'CARGA.EscalaManifesto',
        'localField': 'escala',
        'foreignField': 'escala',
        'as': 'lista_manifestos'
    }}, 
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'lista_manifestos.manifesto': 1}},
    {'$lookup': {
        'from': 'CARGA.Manifesto',
        'localField': 'lista_manifestos.manifesto',
        'foreignField': 'manifesto',
        'as': 'lista_manifestos2'
    }}, 
    {'$match': {'lista_manifestos2.tipomanifesto':
                {'$in': ['lci', 'bce']}}},
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'lista_manifestos2.manifesto': 1}},
    {'$lookup': {
        'from': 'CARGA.ManifestoConhecimento',
        'localField': 'lista_manifestos2.manifesto',
        'foreignField': 'manifesto',
        'as': 'conhecimentos'
    }},
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'conhecimentos.conhecimento': 1,
                  'conhecimentos.manifesto': 1}},
    {'$lookup': {
        'from': 'CARGA.Container',
        'localField': 'conhecimentos.conhecimento',
        'foreignField': 'conhecimento',
        'as': 'containers'
    }},
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'conhecimentos.conhecimento': 1,
                  'conhecimentos.manifesto': 1,
                  'containers.conhecimento': 1,
                  'containers.container': 1
                 }}

]


In [14]:
cursor = db['CARGA.AtracDesatracEscala'].aggregate(AGREGATE_IMPORTACAO)

In [15]:
lista = list(cursor)
len(lista)

175

In [16]:
lista[0]

{'_id': ObjectId('5c5342e31004b333d727dcea'),
 'conhecimentos': [{'conhecimento': '151707128765934',
   'manifesto': '1517701441714'},
  {'conhecimento': '151707129619575', 'manifesto': '1517701449049'},
  {'conhecimento': '151707129619656', 'manifesto': '1517701449049'},
  {'conhecimento': '151707129619737', 'manifesto': '1517701449049'},
  {'conhecimento': '151707129619818', 'manifesto': '1517701449049'},
  {'conhecimento': '151707129619907', 'manifesto': '1517701449049'},
  {'conhecimento': '151707129620077', 'manifesto': '1517701449049'},
  {'conhecimento': '151707129620310', 'manifesto': '1517701449057'},
  {'conhecimento': '151707129620409', 'manifesto': '1517701449065'},
  {'conhecimento': '151707129620581', 'manifesto': '1517701449065'},
  {'conhecimento': '151707129620662', 'manifesto': '1517701449065'},
  {'conhecimento': '151707129620743', 'manifesto': '1517701449065'},
  {'conhecimento': '151707129620824', 'manifesto': '1517701449073'},
  {'conhecimento': '151707129620905',

In [17]:
for conhecimento in lista[0]['containers']:
    print(conhecimento)

{'conhecimento': '001709022013865', 'container': 'medu2079120'}
{'conhecimento': '001709022013865', 'container': 'medu2298767'}
{'conhecimento': '001709022013865', 'container': 'medu2991030'}
{'conhecimento': '001709022013865', 'container': 'mscu6709725'}
{'conhecimento': '001709022013946', 'container': 'medu2314636'}
{'conhecimento': '001709022013946', 'container': 'sczu7944560'}
{'conhecimento': '001709022014080', 'container': 'clhu2656005'}
{'conhecimento': '001709022014080', 'container': 'fscu3242959'}
{'conhecimento': '001709022014080', 'container': 'tcku3399322'}
{'conhecimento': '001709022014160', 'container': 'caxu4716851'}
{'conhecimento': '001709022014160', 'container': 'mscu4840839'}
{'conhecimento': '001709022014160', 'container': 'mscu5010224'}
{'conhecimento': '001709022014160', 'container': 'texu7393059'}
{'conhecimento': '001709022014241', 'container': 'mscu4767023'}
{'conhecimento': '001709022014241', 'container': 'mscu4783050'}
{'conhecimento': '001709022014241', 'con

In [18]:
set_conhecimentos = set()
dict_conhecimentos = dict()
for lista2 in lista:
    for linha in lista2['containers']:
        dict_conhecimentos[linha['container']] = linha['conhecimento']
        set_conhecimentos.add(linha['container'])
        
len(set_conhecimentos)

60772

In [19]:
encontrados_importacao = set_faltantes & set_conhecimentos
encontrados_importacao

{'amfu8765746',
 'beau2213110',
 'bmou2679382',
 'caiu8203840',
 'caxu6243130',
 'clhu3052150',
 'etnu1501715',
 'etnu1501802',
 'etnu1501850',
 'etnu1501865',
 'etnu1502049',
 'hasu1513262',
 'hasu4586771',
 'hlbu1631941',
 'lcru0146847',
 'medu1149223',
 'medu3100670',
 'medu3112500',
 'mscu5950018',
 'mscu6402390',
 'sudu5965453',
 'sudu7727412',
 'tghu2692604',
 'ttnu0704625',
 'ttnu4881681',
 'ttnu5169835',
 'uacu3711340',
 'uacu8063018'}

In [20]:
AGREGATE_EXPORTACAO = [
    {'$match': {'dataatracacaoiso':
                {'$gte': start  - timedelta(days=1),
                 '$lte': end + timedelta(days=DELTA_EXPORTACAO)}}},
    {'$lookup': {
        'from': 'CARGA.EscalaManifesto',
        'localField': 'escala',
        'foreignField': 'escala',
        'as': 'lista_manifestos'
    }}, 
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'lista_manifestos.manifesto': 1}},
    {'$lookup': {
        'from': 'CARGA.Manifesto',
        'localField': 'lista_manifestos.manifesto',
        'foreignField': 'manifesto',
        'as': 'lista_manifestos2'
    }}, 
    {'$match': {'lista_manifestos2.tipomanifesto':
                {'$in': ['lce']}}},
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'lista_manifestos2.manifesto': 1}},
    {'$lookup': {
        'from': 'CARGA.ManifestoConhecimento',
        'localField': 'lista_manifestos2.manifesto',
        'foreignField': 'manifesto',
        'as': 'conhecimentos'
    }},
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'conhecimentos.conhecimento': 1,
                  'conhecimentos.manifesto': 1}},
    {'$lookup': {
        'from': 'CARGA.Container',
        'localField': 'conhecimentos.conhecimento',
        'foreignField': 'conhecimento',
        'as': 'containers'
    }},
    {'$project': {'escala':1, 'dataatracacaoiso': 1,
                  'conhecimentos.conhecimento': 1,
                  'conhecimentos.manifesto': 1,
                  'containers.conhecimento': 1,
                  'containers.container': 1
                 }}

]


In [21]:
cursor = db['CARGA.AtracDesatracEscala'].aggregate(AGREGATE_EXPORTACAO)

In [22]:
lista = list(cursor)
len(lista)


238

In [23]:
set_conhecimentos = set()
dict_conhecimentos = dict()
for lista2 in lista:
    for linha in lista2['containers']:
        dict_conhecimentos[linha['container']] = linha['conhecimento']
        set_conhecimentos.add(linha['container'])
        
len(set_conhecimentos)

64363

In [24]:
encontrados_exportacao = set_faltantes & set_conhecimentos
encontrados_exportacao

{'amfu8682512',
 'amfu8765746',
 'bhcu3176527',
 'bhcu3204064',
 'bmou4529550',
 'bmou5830105',
 'cadu7040225',
 'caxu9342076',
 'clhu3850843',
 'cniu1185248',
 'cpsu1805229',
 'crxu8688161',
 'dfsu2364962',
 'dhdu1375757',
 'exfu0721671',
 'fciu2311362',
 'fciu4264001',
 'fciu8627770',
 'fscu3686980',
 'fscu6890359',
 'fscu9270860',
 'fscu9604741',
 'gesu3844463',
 'hasu1018417',
 'hasu1431354',
 'hasu1459217',
 'hasu1513262',
 'hasu1545125',
 'hasu4045972',
 'hasu4053860',
 'hasu4208766',
 'hasu4252748',
 'hasu4419331',
 'hasu4473760',
 'hasu4560503',
 'hlbu1631941',
 'hlbu1698040',
 'hlxu6464171',
 'hlxu8263788',
 'hlxu8314740',
 'hlxu8348864',
 'hlxu8442171',
 'ibtu1251685',
 'ihou1840712',
 'mscu6322401',
 'rfcu8211880',
 'rfcu8229344',
 'rltu2016777',
 'segu5627986',
 'sudu1481872',
 'sudu1920430',
 'sudu4869630',
 'sudu4935120',
 'sudu5163313',
 'sudu5796097',
 'sudu5848966',
 'sudu5893477',
 'sudu5965453',
 'sudu6053270',
 'sudu6555854',
 'sudu6593746',
 'sudu6764978',
 'sudu68

In [25]:
encontrados_exportacao & encontrados_vazio

{'triu0300295'}

In [27]:
encontrados_exportacao & encontrados_importacao

{'amfu8765746',
 'hasu1513262',
 'hlbu1631941',
 'sudu5965453',
 'sudu7727412',
 'ttnu0704625',
 'ttnu5169835'}

In [28]:
encontrados_vazio & encontrados_importacao

set()